# Instruction:

1. Upload "submission.zip" to Colab.
2. Upload your "truetest_data.mat" to Colab.
2. Press "Runtime" -> "Run all". Leave the window open and active for 8-10 minutes until a green tick shows up on the upper right corner next to "RAM" and "Disk", indicating that the run is complete.
3. Refresh Colab Files. Download "predictions.mat" as the final output file for the true test data.

# Data Preparation

In [1]:
%%capture
!unzip 'submission.zip'

In [2]:
import scipy
import pickle
import numpy as np
import pandas as pd
import lightgbm as lgb
import matplotlib.pyplot as plt
from scipy import signal as sig
from scipy.stats import pearsonr
from scipy.interpolate import CubicSpline
from sklearn.preprocessing import StandardScaler
from joblib import dump, load

## Load Data

In [ ]:
data = scipy.io.loadmat('truetest_data.mat')
data = [data['truetest_data'][i][0] for i in range(3)]

In [ ]:
N = data[0].shape[0]

## Filter Data

In [ ]:
def filter_data(raw_eeg, fs=1000):
    sos = sig.butter(8, [0.15,200],btype='bandpass',output='sos',fs=fs)
    clean_data = sig.sosfiltfilt(sos, raw_eeg, axis=0)
    return clean_data

## Get Features

In [ ]:
def NumWins(length, fs, winLen, winDisp):
    return round((length/fs-winLen+winDisp)/winDisp)

def get_features(filtered_window, fs=1000):
    return np.vstack([
        np.apply_along_axis(LL, 0, filtered_window),
        np.apply_along_axis(E, 0, filtered_window),
        np.apply_along_axis(BP, 0, filtered_window, f_min=75, f_max=115),
        np.apply_along_axis(BP, 0, filtered_window, f_min=125, f_max=159),
        np.apply_along_axis(BP, 0, filtered_window, f_min=159, f_max=175),
    ]).T

def get_windowed_feats(raw_ecog, fs, window_length, window_overlap):
    data = filter_data(raw_ecog)
    n = NumWins(len(data), fs, window_length, window_overlap)
    windows = []
    for i in range(n):
        start = round(i*window_overlap*fs)
        windows.append(get_features(data[start:start+round(window_length*fs),:]))
    return np.array(windows)

# Line Length
def LL(x):
    return np.sum(np.abs(np.ediff1d(x)))

# Energy
def E(x):
    return np.sum(np.square(x))

# Bandpower
def BP(x, f_min, f_max, fs=1000):
    f, Pxx = sig.periodogram(x, fs=fs)
    i_min = np.argmax(f>f_min)-1
    i_max = np.argmax(f>f_max)-1
    return np.trapz(Pxx[i_min:i_max], f[i_min:i_max])

In [ ]:
# RUNTIME WARNING: 7 MIN
X_test = [get_windowed_feats(data[i], 1000, 0.1, 0.05) for i in range(3)]

In [ ]:
for i in range(3):
    X_test[i] = X_test[i].reshape(X_test[i].shape[0], -1)

## Normalize Features

In [ ]:
scalers = [load(f'scaler{i+1}.bin') for i in range(3)]
X_test = [scalers[i].transform(X_test[i]) for i in range(3)]

# Prediction

## Load Models

In [ ]:
regs = []
for i in range(3):
    regs_subj = []
    for j in range(5):
        regs_subj.append(lgb.Booster(model_file=f'lgb_subj{i}_finger{j}.txt'))
    regs.append(regs_subj)

## Predict

In [ ]:
Y_pred = [None]*3
for i in range(3):
    Y_pred[i] = []
    for j in range(5):
        Y_pred[i].append(regs[i][j].predict(X_test[i], num_iteration=regs[i][j].best_iteration))
    Y_pred[i] = np.hstack([vec.reshape(-1,1) for vec in Y_pred[i]])

Y_pred_intped = [np.array([CubicSpline(np.arange(len(Y_pred[i])), Y_pred[i][:,j], bc_type="natural")(np.linspace(0, len(Y_pred[i]), N)) for j in range(5)]).T for i in range(3)]

## Save

In [ ]:
predictions = np.zeros((3,1), dtype=object)
predictions[0,0] = Y_pred_intped[0]
predictions[1,0] = Y_pred_intped[1]
predictions[2,0] = Y_pred_intped[2]

scipy.io.savemat("predictions.mat", {"predicted_dg": predictions})